In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, m, latent_dim=10):
        """
        Variational Autoencoder for learning the distribution of N arrays.

        Args:
            m (int): Number of columns in matrix M (same as the length of array N).
            latent_dim (int, optional): Dimension of the latent space. Defaults to 10.
        """
        super(VAE, self).__init__()
        self.m = m
        self.latent_dim = latent_dim

        # Encoder layers
        self.fc1 = nn.Linear(m, 128)
        self.fc2_mu = nn.Linear(128, latent_dim)
        self.fc2_logvar = nn.Linear(128, latent_dim)

        # Decoder layers
        self.fc3 = nn.Linear(latent_dim, 128)
        self.fc4 = nn.Linear(128, m)

        # Polynomial Regression Parameter G
        self.G = nn.Parameter(torch.randn(3, m))

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        mu = self.fc2_mu(h1)
        logvar = self.fc2_logvar(h1)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        """
        Reparameterization trick to sample from N(mu, var) from N(0,1).
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        N = self.decode(z)
        # Compute predictions using G
        N_poly = torch.stack([
            torch.ones_like(N),
            N,
            N ** 2
        ], dim=0)  # Shape: (3, batch_size, m)
        G_expanded = self.G.unsqueeze(1)  # Shape: (3, 1, m)
        predictions = (G_expanded * N_poly).sum(dim=0)  # Shape: (batch_size, m)
        return predictions, mu, logvar, N

In [2]:
import torch
from torch.utils.data import Dataset

class PolynomialVaeDataset(Dataset):
    def __init__(self, M_list):
        """
        Custom Dataset for VAE-based polynomial regression.

        Args:
            M_list (list of torch.Tensor): List of M matrices, each of shape (n, m).
        """
        self.M_list = M_list  # List of tensors with shape (n, m)

    def __len__(self):
        return len(self.M_list)

    def __getitem__(self, idx):
        # For VAEs, we can use the mean of M as the target
        return self.M_list[idx].mean(dim=0)

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader


def loss_function(recon_x, x, mu, logvar):
    """
    Computes the VAE loss function as a combination of reconstruction loss and KL divergence.

    Args:
        recon_x (torch.Tensor): Reconstructed M means.
        x (torch.Tensor): Original M means.
        mu (torch.Tensor): Mean from the encoder.
        logvar (torch.Tensor): Log variance from the encoder.

    Returns:
        torch.Tensor: Combined loss.
    """
    mse_loss = nn.MSELoss()(recon_x, x)
    # KL Divergence
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return mse_loss + KLD

def train_vae_model(M_list, n, m, latent_dim=10, batch_size=32, epochs=1000, lr=1e-3):
    """
    Trains the VAE model to learn the distribution of N arrays and G parameters.

    Args:
        M_list (list of torch.Tensor): List of M matrices, each of shape (n, m).
        n (int): Number of rows in each M matrix.
        m (int): Number of columns in each M matrix (same as length of N arrays).
        latent_dim (int, optional): Dimension of the latent space. Defaults to 10.
        batch_size (int, optional): Batch size for training. Defaults to 32.
        epochs (int, optional): Number of training epochs. Defaults to 1000.
        lr (float, optional): Learning rate. Defaults to 1e-3.

    Returns:
        VAE: Trained VAE model.
    """
    dataset = PolynomialVaeDataset(M_list)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VAE(m, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        for batch in dataloader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon, mu, logvar, N = model(batch)
            loss = loss_function(recon, batch, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if epoch % 100 == 0 or epoch == 1:
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch [{epoch}/{epochs}], Loss: {avg_loss:.6f}')

    return model

def generate_multiple_solutions(model, num_samples=5):
    """
    Generates multiple N and G solutions from the trained VAE model.

    Args:
        model (VAE): Trained VAE model.
        num_samples (int, optional): Number of solutions to generate. Defaults to 5.

    Returns:
        list of tuples: Each tuple contains an N tensor and the corresponding G tensor.
    """
    device = next(model.parameters()).device
    model.eval()
    solutions = []
    with torch.no_grad():
        for _ in range(num_samples):
            z = torch.randn(1, model.latent_dim).to(device)
            N = model.decode(z)  # Shape: (1, m)
            G = model.G  # Shape: (3, m)
            solutions.append((N.squeeze(0).cpu(), G.detach().cpu()))
    return solutions

if __name__ == "__main__":
    # Example usage

    # Define dimensions
    n, m = 4, 3  # For example, 4x3 matrices and 3-dimensional N arrays

    # Create example M matrices (without knowing N)
    M_list = [
        torch.tensor([
            [1.0, 2.0, 3.0],
            [2.0, 4.0, 6.0],
            [3.0, 6.0, 9.0],
            [4.0, 8.0, 12.0]
        ]),
        torch.tensor([
            [2.0, 3.0, 4.0],
            [4.0, 6.0, 8.0],
            [6.0, 9.0, 12.0],
            [8.0, 12.0, 16.0]
        ]),
        torch.tensor([
            [3.0, 4.0, 5.0],
            [6.0, 8.0, 10.0],
            [9.0, 12.0, 15.0],
            [12.0, 16.0, 20.0]
        ])
    ]

    # Train the VAE model
    trained_vae = train_vae_model(M_list, n, m, latent_dim=10, batch_size=2, epochs=1000, lr=1e-3)

    # Generate multiple solutions
    num_solutions = 3  # Specify how many solutions you want
    solutions = generate_multiple_solutions(trained_vae, num_samples=num_solutions)

    for idx, (N, G) in enumerate(solutions):
        print(f"\n--- Solution {idx+1} ---")
        print("Generated N array:")
        print(N)
        print("\nLearned parameters G:")
        print(G)

Epoch [1/1000], Loss: 121.771523
Epoch [100/1000], Loss: 20.060699
Epoch [200/1000], Loss: 28.634759
Epoch [300/1000], Loss: 9.888518
Epoch [400/1000], Loss: 6.115332
Epoch [500/1000], Loss: 9.561793
Epoch [600/1000], Loss: 3.156965
Epoch [700/1000], Loss: 2.499913
Epoch [800/1000], Loss: 3.814348
Epoch [900/1000], Loss: 2.437309
Epoch [1000/1000], Loss: 3.260542

--- Solution 1 ---
Generated N array:
tensor([ 3.9949, -2.1573, -2.8973])

Learned parameters G:
tensor([[ 0.4030,  2.1078, -0.4830],
        [ 0.7256, -2.4330, -1.4904],
        [ 0.0470, -0.1597,  0.5669]])

--- Solution 2 ---
Generated N array:
tensor([ 4.4981, -2.4991, -3.0916])

Learned parameters G:
tensor([[ 0.4030,  2.1078, -0.4830],
        [ 0.7256, -2.4330, -1.4904],
        [ 0.0470, -0.1597,  0.5669]])

--- Solution 3 ---
Generated N array:
tensor([ 7.1132, -4.6102, -3.6999])

Learned parameters G:
tensor([[ 0.4030,  2.1078, -0.4830],
        [ 0.7256, -2.4330, -1.4904],
        [ 0.0470, -0.1597,  0.5669]])


In [ ]:
generate_multiple_solutions(trained_vae, num_samples=num_solutions)